<a href="https://colab.research.google.com/github/Ritwik-1/Fine-Tune-SAM/blob/master/Adrenal_Dice_Scores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Segmentations are only for the venous part

In [ ]:
pip install pydicom

In [ ]:
pip install git+https://github.com/facebookresearch/segment-anything.git

In [ ]:
import pydicom

In [1]:
import os
import cv2
import torch
import numpy as np
from scipy.ndimage import zoom
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry

model_checkpoint = "/content/drive/MyDrive/Colab Notebooks/assets/sam_vit_h_4b8939.pth"
model_type = "vit_h"
sam = sam_model_registry[model_type](checkpoint = model_checkpoint)
sam.to(device)

from segment_anything import SamPredictor

predictor = SamPredictor(sam)

In [ ]:
CT_Images = []
Seg_Labels = []
Bounding_Boxes = []

In [2]:
# for each patient take venous images and segmentations and bounding boxes
# 10 patients now and around 1k images
# superimpose image + provided seg and image + sam seg

above_patients_path = "/content/drive/MyDrive/manifest-1680809675630/Adrenal-ACC-Ki67-Seg"

patients = os.listdir(above_patients_path)

for patient in patients:
    print("Patient ID : ",patient)
    # patient = Adrenal_Ki67_Seg_001
    patient_folder_path = os.path.join(above_patients_path,patient)

    # patient folder = 08-22-2000-NA-CT ABDOMEN-56266
    patient_folder = os.listdir(patient_folder_path)[0]

    patient_filesfolder_path = os.path.join(patient_folder_path,patient_folder)

    # patient_filesfolder is the list which has 2-3 folders which
    # are the venous,non-venous,segmentation etc
    patient_filesfolder = os.listdir(patient_filesfolder_path)

    print("Folders :-")
    for folder in patient_filesfolder:
      print(folder)
    print()




Patient ID :  Adrenal_Ki67_Seg_051
Folders :-
401.000000-ABD AX ART iDose 4-12159
300.000000-Segmentation-43028

Patient ID :  Adrenal_Ki67_Seg_052
Folders :-
302.000000-ABD AX 1.25 PRE-17123
20.000000-ABD PEL AX 1.25 DEL 15-01130

Patient ID :  Adrenal_Ki67_Seg_046
Folders :-
300.000000-Segmentation-68934
4.000000-No series description-14573

Patient ID :  Adrenal_Ki67_Seg_047
Folders :-
300.000000-Segmentation-06609
2.000000-ABDPELV SFT TISSUE 3X3-62201

Patient ID :  Adrenal_Ki67_Seg_053
Folders :-
12.000000-ABD AX 3 DEL-25085
7.000000-ABD AX 3 PV-01373
300.000000-Segmentation-13639
3.000000-ABD AX 3 PRE-55155

Patient ID :  Adrenal_Ki67_Seg_048
Folders :-
2.000000-NA-94141
300.000000-Segmentation-72337

Patient ID :  Adrenal_Ki67_Seg_050
Folders :-
300.000000-Segmentation-70908
201.000000-ABDOMENPELVIS W iDose 3-36045

Patient ID :  Adrenal_Ki67_Seg_045
Folders :-
2.000000-CHESTABD WITH-35882
6.000000-DELAY-18500
300.000000-Segmentation-11611

Patient ID :  Adrenal_Ki67_Seg_049
Fol